In [1]:
# Run this to ensure TensorFlow 2.x is used
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [7]:
import json
import tensorflow as tf
import pandas as pd


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
df = pd.read_csv('/content/drive/My Drive/Test.csv')

In [9]:
df.head()

,text,label
0,I always wrote this series off as being a comp...,0
1,1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...,0
2,This movie was so poorly written and directed ...,0
3,The most interesting thing about Miryang (Secr...,1
4,"when i first read about ""berlin am meer"" i did...",0


In [10]:
df.shape

(5000, 2)

In [12]:
df['text']

0       I always wrote this series off as being a comp...
1       1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...
2       This movie was so poorly written and directed ...
3       The most interesting thing about Miryang (Secr...
4       when i first read about "berlin am meer" i did...
                              ...                        
4995    This is the kind of picture John Lassiter woul...
4996    A MUST SEE! I saw WHIPPED at a press screening...
4997    NBC should be ashamed. I wouldn't allow my chi...
4998    This movie is a clumsy mishmash of various gho...
4999    Formula movie about the illegitimate son of a ...
Name: text, Length: 5000, dtype: object

In [17]:
vocab_size = 4000
embedding_dim = 16
max_length = 90
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 4000

In [18]:
training_sentences = df.iloc[:training_size]['text']

In [ ]:
training_sentences

In [22]:
testing_sentences = df.iloc[training_size:]['text']

In [ ]:
testing_sentences

In [24]:
training_labels = df.iloc[:training_size]['label']


In [ ]:
training_labels

In [26]:
testing_labels = df.iloc[training_size:]['label']

In [ ]:
testing_labels

In [29]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
training_sentences_sequences = tokenizer.texts_to_sequences(training_sentences)
training_sentences_padded = pad_sequences(training_sentences_sequences)

In [30]:
testing_sentences_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_sentences_padded = pad_sequences(testing_sentences_sequences)

In [32]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                             tf.keras.layers.GlobalAveragePooling1D(),
                             tf.keras.layers.Dense(24, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 90, 16)            64000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                408       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 64,433
Trainable params: 64,433
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
num_epochs = 30 


In [38]:
import numpy as np
training_padded = np.array(training_sentences_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_sentences_padded)
testing_labels = np.array(testing_labels)

In [39]:
history = model.fit(training_sentences_padded, training_labels, epochs=num_epochs, validation_data=(testing_sentences_padded, testing_labels), verbose=2)

Epoch 1/30
125/125 - 1s - loss: 0.1259 - accuracy: 0.9605 - val_loss: 0.4135 - val_accuracy: 0.8760
Epoch 2/30
125/125 - 1s - loss: 0.1211 - accuracy: 0.9640 - val_loss: 0.4341 - val_accuracy: 0.8800
Epoch 3/30
125/125 - 1s - loss: 0.1193 - accuracy: 0.9650 - val_loss: 0.4320 - val_accuracy: 0.8750
Epoch 4/30
125/125 - 1s - loss: 0.1108 - accuracy: 0.9675 - val_loss: 0.4411 - val_accuracy: 0.8730
Epoch 5/30
125/125 - 1s - loss: 0.1044 - accuracy: 0.9697 - val_loss: 0.4536 - val_accuracy: 0.8710
Epoch 6/30
125/125 - 1s - loss: 0.1012 - accuracy: 0.9725 - val_loss: 0.4608 - val_accuracy: 0.8730
Epoch 7/30
125/125 - 1s - loss: 0.0965 - accuracy: 0.9735 - val_loss: 0.4791 - val_accuracy: 0.8700
Epoch 8/30
125/125 - 1s - loss: 0.0950 - accuracy: 0.9743 - val_loss: 0.4816 - val_accuracy: 0.8700
Epoch 9/30
125/125 - 1s - loss: 0.0876 - accuracy: 0.9755 - val_loss: 0.5015 - val_accuracy: 0.8730
Epoch 10/30
125/125 - 1s - loss: 0.0825 - accuracy: 0.9785 - val_loss: 0.5054 - val_accuracy: 0.8680

In [42]:
sentence = [
            'Sholay is a grand, maximalist revel. It''s the kind of movie that, while it may pass into the status of so many older movies more talked about than seen, it became that thing by being great.',
            'I''m an IITian myself and hence, needless to say, was looking forward to this movie as it is based on the life in IITs. I went with my family and watched the first-day-first-show and was I pleased ? In one sentence - The best Bollywood movie ever ! If you are into movies, then this one is not to be missed unless you are the 4th idiot. I cried, I laughed and I enjoyed every moment of the 3 hours that I spent watching this gem. The songs that seemed mediocre before watching the movie, feel like perfect for the movie. They are so ideal for the situations that I just loved them. As you can understand, they could not bring the name of any IIT into the movie to avoid disputes. But trust me, whatever they have shown about the life in IITs, it''s all true. Right from suicides, cold-hearted Professors to lack of encouragement and support for talents who can really bring serious change in this world. The movie successfully depicts how once inside, your life is only about meaningless grades, go get into the race before it''s too late. Take my free advice and avoid any reviews or words from people who have already seen the movie. Don''t let anyone spoil any scene from the movie. Rush to the nearest theater and experience the phenomenon for yourself. You''ll bless me for this advice !',
            'Now onto what I didn''t like about this film. Salman''s love actress, although beautiful, has an insignificant role in this movie. '
]

sequence = tokenizer.texts_to_sequences(sentence)
padding = pad_sequences(sequence)

print(model.predict(padding))

[[1.000000e+00]
 [2.065728e-08]
 [9.999970e-01]]
